In [15]:
import requests
from bs4 import BeautifulSoup
from bs4.element import Doctype, ResultSet
#import ngram

def getResponse(url, type = 'bs4'):
    context = None
    data={}
    headers= {'Cache-Control': 'no-cache',
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                'Accept-Language': 'pt-BR,pt;q=0.8,en-US;q=0.6,en;q=0.4'
            }

    response = requests.get(url, data=data, headers=headers)
    if response is not None:

        if type == 'bs4':
            response.raise_for_status()
            context = BeautifulSoup(response.text, 'lxml')
            
            dtd = next(context.descendants)
            try:            
                if type(dtd) is Doctype:
                    dtd.extract()
            except:
                dtd = None
        elif type == "binary":
            #context = BytesIO(response.content)
            context = response.content
            
        else:
            context = response.text
    
    return context


In [16]:
from io import BytesIO
import pickle
import sys
import random

def savePickleFile(filename, rule):
    print("> saving file:", filename)
    f = open(filename,"wb")
    pickle.dump(rule,f)
    f.close()
    print("> saved")
    
def loadPickleFile(filename):    
    print("> loading file:", filename)
    file = open(filename, 'rb')
    data = pickle.load(file)
    file.close()
    print("> loaded")
    return data


In [17]:
def processMenu(resp, ns, linkQueue = [], linkDone = []):

    menuLinks = [] 
    dataBox = resp.find("nav", id="menu-container")
    if dataBox != None:
        for link in dataBox.find_all("a",class_="menu-item-link"):
            if 'href' in link.attrs and type(link.attrs['href']) == str:
                url = link.attrs['href']
                url = url.replace('http:','https:')
                if url.startswith(flags['baseUrl']) and \
                    url not in linkQueue and \
                    url not in menuLinks: 
                    # url not in linkDone and \                
                    menuLinks.append(url)
                    #print(url)
    return menuLinks

def processNews(resp, ns, linkQueue = [], linkDone = []):
    headline = ""
    article = ""
    category = ""
    articleLinks = [] 
    
    oTitle = resp.find("h1",class_="content-head__title")
    if oTitle != None:
        headline = oTitle.text
    
    for dataBox in resp.find_all("p",class_="content-text__container"):
        for link in dataBox.find_all("a"):
            if 'href' in link.attrs and type(link.attrs['href']) == str:
                url = link.attrs['href']
                url = url.replace('http:','https:')
                if url.startswith(flags['baseUrl']) and \
                    url not in linkQueue and \
                    url not in linkDone and \
                    url not in articleLinks: 
                    articleLinks.append(url)
        article += dataBox.text + "\n"
    
    dataBox = resp.find("div",class_="area-principal")
    if dataBox != None:
        for link in dataBox.find_all("a",class_="header-editoria--link"):
            if link != None and link.text != None:
                category = link.text.lower().strip()

    article = article.replace('\r','')
    while article.find('\t') > -1:
        article = article.replace('\t',' ')    
    while article.find('  ') > -1:
        article = article.replace('  ',' ')
    while article.find('\n ') > -1:
        article = article.replace('\n ','\n')        
    while article.find('\n\n') > -1:
        article = article.replace('\n\n','\n')

    article = article.strip('\n')
    article = article.strip()

    return headline, article, category, articleLinks


def processLinks(resp, ns, linkQueue = [], linkDone = []):
    articleLinks = [] 

    for dataBox in resp.find_all("div",class_="feed-post"):
        for link in dataBox.find_all("a"):
            if 'href' in link.attrs and type(link.attrs['href']) == str:
                url = link.attrs['href']
                url = url.replace('http:','https:')
                if url.startswith(flags['baseUrl']) and \
                    url not in linkQueue and \
                    url not in linkDone and \
                    url not in articleLinks: 
                    articleLinks.append(url)
    return articleLinks

def processQueue(flags, ns, rule2, rule3, linkQueue, linkDone, procName=""):
    
    try:
        queueLock.acquire()
        #i = i + 1
        flags['i'] = flags['i'] + 1
        url = linkQueue.pop(random.randint(0, len(linkQueue)-1))
        queueLock.release()

        print("----------------------------")
        print(procName + " # processing... " + url)
        if any(b for b in ns.news_df['news_url'].str.contains(url, regex=False)):
            print("................ in corpus")
            if url.endswith('/'):
                print("... reading links")
                resp = getResponse(url)
                otherLinks = processLinks(resp, ns, linkQueue, linkDone)
                queueLock.acquire()
                linkQueue.extend(otherLinks)
                queueLock.release()

        else:
            resp = getResponse(url)

            headline, article, category, articleLinks = processNews(resp, ns, linkQueue, linkDone)
            otherLinks = processLinks(resp, ns, linkQueue, linkDone)

            #queueLock.acquire()
            #if article.strip != '':
            #    rule2 = makeRule(article, 2, rule2)
            #    rule3 = makeRule(article, 3, rule3)        
            #queueLock.release()

            queueLock.acquire()
            news_df = ns.news_df
            news_df.loc[len(news_df)] = {'news_url': url, 'news_headline':headline, 'news_article':article,'clean_text':'', 'news_category':category} 
            ns.news_df = news_df
            linkQueue.extend(articleLinks)
            linkQueue.extend(otherLinks)
            queueLock.release()

        queueLock.acquire()
        linkDone.append(url)
        queueLock.release()

        showStatus(ns, rule2, rule3, linkQueue, linkDone)
        
        #if flags['i'] >= 100:
        #    flags['i'] = 0
        #    savingFiles(ns, rule2, rule3, linkQueue, linkDone)

        print()
        
    except Exception as ex:
        #queueLock.release()
        print(traceback.format_exc())

def showStatus(ns, rule2, rule3, linkQueue, linkDone):
    print("Queue size:", len(linkQueue))
    print("Done size:", len(linkDone))
    print("News size:", len(ns.news_df))

def savingFiles(ns, rule2, rule3, linkQueue, linkDone):
    print("Saving files...")
    savePickleFile("news_df.pkl", ns.news_df.copy())
    savePickleFile("memoryLinksQueue.pkl", linkQueue[:])
    savePickleFile("memoryLinksDone.pkl", linkDone[:])
    print("Saved!")

 ################################################################       

In [18]:
ns.news_df

,news_url,news_headline,news_article,clean_text,news_category


In [19]:
linkDone = []
linkQueue = []
rule = {}
context = 3

In [20]:
#url = "https://g1.globo.com/mundo/noticia/2019/12/22/militar-morre-em-ataque-a-quartel-no-sul-da-venezuela.ghtml"
url = "https://g1.globo.com/"
linkQueue.append(url)



In [22]:
import queue
import threading
import pandas as pd
import traceback
import random
from multiprocessing import Pool, Manager

news_df = None
linkQueue = None
linkDone = None

queueLock = threading.Lock()

# Load variables
rule2 = None
rule3 = None
print("Loading memory files...")
#news_df = loadPickleFile("news_df.pkl")
#linkQueue = loadPickleFile("memoryLinksQueue.pkl")
#linkDone = loadPickleFile("memoryLinksDone.pkl")
#l3 = [x for x in linkQueue if x not in linkDone]
#linkQueue = l3

if not isinstance(news_df, pd.DataFrame):
    news_df = pd.DataFrame(columns =['news_url', 'news_headline', 'news_article','clean_text', 'news_category'])
if linkDone == None:
    linkDone = []
if linkQueue == None:
    linkQueue = []
###########

# Create managed variables
manager = Manager()
flags = manager.dict({'exitFlag' : 0, 'i': 0, 'baseUrl': "https://g1.globo.com/"})
ns = manager.Namespace()
ns.news_df = news_df
linkQueue = manager.list(linkQueue)
linkDone = manager.list(linkDone)
###########

#showStatus(ns, rule2, rule3, linkQueue, linkDone)

if len(linkQueue) == 0:
    linkQueue.append(flags['baseUrl'])

    print("Loading menu...")
    resp = getResponse(flags['baseUrl'])
    linkQueue.extend(processMenu(resp, ns, linkQueue, linkDone))

while len(linkQueue) > 0:
    processQueue(flags, ns, rule2, rule3, linkQueue, linkDone)

Loading memory files...
Loading menu...
----------------------------
 # processing... https://g1.globo.com/rr/roraima/
Queue size: 285
Done size: 1
News size: 1

----------------------------
 # processing... https://g1.globo.com/sp/presidente-prudente-regiao/
Queue size: 290
Done size: 2
News size: 2

----------------------------
 # processing... https://g1.globo.com/rj/rio-de-janeiro/bom-dia-rio/
Queue size: 299
Done size: 3
News size: 3

----------------------------
 # processing... https://g1.globo.com/mg/triangulo-mineiro/
Queue size: 305
Done size: 4
News size: 4

----------------------------
 # processing... https://g1.globo.com/mg/minas-gerais/pelas-cozinhas-de-minas/
Queue size: 312
Done size: 5
News size: 5

----------------------------
 # processing... https://g1.globo.com/mg/minas-gerais/pelas-cozinhas-de-minas/noticia/2018/08/17/pelas-cozinhas-de-minas-mostra-as-abelhas-que-estao-ganhando-fama-no-mercado-exterior.ghtml
Queue size: 327
Done size: 6
News size: 6

------------

Queue size: 547
Done size: 50
News size: 49

----------------------------
 # processing... https://g1.globo.com/economia/educacao-financeira/
Queue size: 553
Done size: 51
News size: 50

----------------------------
 # processing... https://g1.globo.com/profissao-reporter/
Queue size: 561
Done size: 52
News size: 51

----------------------------
 # processing... https://g1.globo.com/rr/roraima/noticia/2023/10/07/suspeito-de-assalto-e-atingido-com-golpes-de-faca-durante-roubo-a-acougueiro-em-boa-vista.ghtml
Queue size: 561
Done size: 53
News size: 52

----------------------------
 # processing... https://g1.globo.com/jornal-nacional/50-anos/
Queue size: 567
Done size: 54
News size: 53

----------------------------
 # processing... https://g1.globo.com/rj/rio-de-janeiro/rio-innovation-week/noticia/2023/10/05/rio-innovation-week-luciano-huck-fala-sobre-democratizacao-da-tecnologia-para-influenciar-o-empreendedorismo.ghtml
Queue size: 566
Done size: 55
News size: 54

----------------------

Queue size: 660
Done size: 87
News size: 85

----------------------------
 # processing... https://g1.globo.com/globonews/podcast/em-movimento/noticia/2019/11/11/em-movimento-13-inovacao-social-atraves-das-artes.ghtml
Queue size: 661
Done size: 88
News size: 86

----------------------------
 # processing... https://g1.globo.com/mg/sul-de-minas/
Queue size: 666
Done size: 89
News size: 87

----------------------------
 # processing... https://g1.globo.com/educacao/guia-de-carreiras/noticia/2019/01/18/fez-o-enem-2018-veja-o-novo-simulador-das-vagas-do-sisu-divulgado-pelo-mec-pela-1a-vez.ghtml
Queue size: 669
Done size: 90
News size: 88

----------------------------
 # processing... https://g1.globo.com/politica/blog/julia-duailibi/post/2023/09/20/moraes-nega-pedido-da-oab-para-que-julgamento-de-acoes-sobre-8-de-janeiro-seja-presencial.ghtml
Queue size: 673
Done size: 91
News size: 89

----------------------------
 # processing... https://g1.globo.com/rj/rio-de-janeiro/rj2/
Queue size: 68

Queue size: 822
Done size: 127
News size: 123

----------------------------
 # processing... https://g1.globo.com/economia/noticia/2023/10/09/nobel-de-economia-2023-vai-para-claudia-goldin.ghtml
Queue size: 826
Done size: 128
News size: 124

----------------------------
 # processing... https://g1.globo.com/rs/rio-grande-do-sul/noticia/2023/05/26/justica-condena-defensores-de-tratamento-precoce-contra-a-covid-19-a-pagar-r-55-milhoes-por-danos-coletivos-e-a-saude.ghtml
Queue size: 831
Done size: 129
News size: 125

----------------------------
 # processing... https://g1.globo.com/rj/rio-de-janeiro/podcast/desenrola-rio/noticia/2023/07/20/desenrola-rio-191-os-lixoes-fluminenses-vao-acabar-quando.ghtml
Queue size: 831
Done size: 130
News size: 126

----------------------------
 # processing... https://g1.globo.com/rj/rio-de-janeiro/playlist/100-anos-da-portela-serie-do-rj1-celebra-o-centenario-azul-e-branco.ghtml
Queue size: 830
Done size: 131
News size: 127

----------------------------

Queue size: 877
Done size: 164
News size: 160

----------------------------
 # processing... https://g1.globo.com/pop-arte/musica/the-town/2023/noticia/2023/08/25/the-town-2023-o-que-pode-e-que-nao-pode-levar-ao-festival.ghtml
Queue size: 876
Done size: 165
News size: 161

----------------------------
 # processing... https://g1.globo.com/rj/rio-de-janeiro/bom-dia-rio/video/faixa-exclusiva-de-onibus-na-avenida-brasil-tem-1-dia-util-de-funcionamento-12012087.ghtml
Queue size: 875
Done size: 166
News size: 162

----------------------------
 # processing... https://g1.globo.com/economia/educacao-financeira/noticia/2023/10/03/entrar-no-rotativo-ou-parcelar-fatura-com-juros-saiba-as-diferencas-e-qual-e-a-opcao-de-credito-mais-cara.ghtml
Queue size: 884
Done size: 167
News size: 163

----------------------------
 # processing... https://g1.globo.com/economia/pme/pequenas-empresas-grandes-negocios/noticia/2020/09/06/aplicativo-ajuda-a-melhorar-atendimento-e-gestao-de-barracas-de-praia.ghtml
Q

KeyboardInterrupt: 

In [38]:
links = []
url = "https://g1.globo.com/"
resp = getResponse(url)
dddd = resp.find("nav", id="menu-container")
for link in dddd.find_all("a",class_="menu-item-link"):
    if 'href' in link.attrs:
        url = link.attrs['href']
        url = url.replace('http:','https:')
        if url.startswith('https://g1.globo.com/') and url not in links:
            print(url)
            links.append(url)

https://g1.globo.com/economia/agronegocios/
https://g1.globo.com/economia/agronegocios/globo-rural/
https://g1.globo.com/economia/agronegocios/agro-a-industria-riqueza-do-brasil/
https://g1.globo.com/bemestar/
https://g1.globo.com/carros/
https://g1.globo.com/carros/a-z/
https://g1.globo.com/carros/caminhoes/
https://g1.globo.com/carros/carros-eletricos-e-hibridos/
https://g1.globo.com/carros/motos/
https://g1.globo.com/carros/assuntos/ipva.html
https://g1.globo.com/carros/tabela-fipe/
https://g1.globo.com/ciencia-e-saude/
https://g1.globo.com/ciencia-e-saude/viva-voce/
https://g1.globo.com/economia/concursos-e-emprego/
https://g1.globo.com/economia/
https://g1.globo.com/economia/seu-dinheiro/calculadoras/
https://g1.globo.com/economia/dolar/
https://g1.globo.com/economia/seu-dinheiro/
https://g1.globo.com/economia/imposto-de-renda/
https://g1.globo.com/economia/tecnologia/inovacao/
https://g1.globo.com/economia/midia-e-marketing/
https://g1.globo.com/economia/pme/
https://g1.globo.com

In [ ]:
import re

corpus = []

total = len(ns.news_df['news_article'])
i = 0

for s in ns.news_df['news_article']: 
    wordList = re.sub("[^\w]", " ",  s).split()
    corpus = corpus + wordList
    i = i+ 1
    print(i,"/",total)

1 / 8222
2 / 8222
3 / 8222
4 / 8222
5 / 8222
6 / 8222
7 / 8222
8 / 8222
9 / 8222
10 / 8222
11 / 8222
12 / 8222
13 / 8222
14 / 8222
15 / 8222
16 / 8222
17 / 8222
18 / 8222
19 / 8222
20 / 8222
21 / 8222
22 / 8222
23 / 8222
24 / 8222
25 / 8222
26 / 8222
27 / 8222
28 / 8222
29 / 8222
30 / 8222
31 / 8222
32 / 8222
33 / 8222
34 / 8222
35 / 8222
36 / 8222
37 / 8222
38 / 8222
39 / 8222
40 / 8222
41 / 8222
42 / 8222
43 / 8222
44 / 8222
45 / 8222
46 / 8222
47 / 8222
48 / 8222
49 / 8222
50 / 8222
51 / 8222
52 / 8222
53 / 8222
54 / 8222
55 / 8222
56 / 8222
57 / 8222
58 / 8222
59 / 8222
60 / 8222
61 / 8222
62 / 8222
63 / 8222
64 / 8222
65 / 8222
66 / 8222
67 / 8222
68 / 8222
69 / 8222
70 / 8222
71 / 8222
72 / 8222
73 / 8222
74 / 8222
75 / 8222
76 / 8222
77 / 8222
78 / 8222
79 / 8222
80 / 8222
81 / 8222
82 / 8222
83 / 8222
84 / 8222
85 / 8222
86 / 8222
87 / 8222
88 / 8222
89 / 8222
90 / 8222
91 / 8222
92 / 8222
93 / 8222
94 / 8222
95 / 8222
96 / 8222
97 / 8222
98 / 8222
99 / 8222
100 / 8222
101 / 82

791 / 8222
792 / 8222
793 / 8222
794 / 8222
795 / 8222
796 / 8222
797 / 8222
798 / 8222
799 / 8222
800 / 8222
801 / 8222
802 / 8222
803 / 8222
804 / 8222
805 / 8222
806 / 8222
807 / 8222
808 / 8222
809 / 8222
810 / 8222
811 / 8222
812 / 8222
813 / 8222
814 / 8222
815 / 8222
816 / 8222
817 / 8222
818 / 8222
819 / 8222
820 / 8222
821 / 8222
822 / 8222
823 / 8222
824 / 8222
825 / 8222
826 / 8222
827 / 8222
828 / 8222
829 / 8222
830 / 8222
831 / 8222
832 / 8222
833 / 8222
834 / 8222
835 / 8222
836 / 8222
837 / 8222
838 / 8222
839 / 8222
840 / 8222
841 / 8222
842 / 8222
843 / 8222
844 / 8222
845 / 8222
846 / 8222
847 / 8222
848 / 8222
849 / 8222
850 / 8222
851 / 8222
852 / 8222
853 / 8222
854 / 8222
855 / 8222
856 / 8222
857 / 8222
858 / 8222
859 / 8222
860 / 8222
861 / 8222
862 / 8222
863 / 8222
864 / 8222
865 / 8222
866 / 8222
867 / 8222
868 / 8222
869 / 8222
870 / 8222
871 / 8222
872 / 8222
873 / 8222
874 / 8222
875 / 8222
876 / 8222
877 / 8222
878 / 8222
879 / 8222
880 / 8222
881 / 8222

1516 / 8222
1517 / 8222
1518 / 8222
1519 / 8222
1520 / 8222
1521 / 8222
1522 / 8222
1523 / 8222
1524 / 8222
1525 / 8222
1526 / 8222
1527 / 8222
1528 / 8222
1529 / 8222
1530 / 8222
1531 / 8222
1532 / 8222
1533 / 8222
1534 / 8222
1535 / 8222
1536 / 8222
1537 / 8222
1538 / 8222
1539 / 8222
1540 / 8222
1541 / 8222
1542 / 8222
1543 / 8222
1544 / 8222
1545 / 8222
1546 / 8222
1547 / 8222
1548 / 8222
1549 / 8222
1550 / 8222
1551 / 8222
1552 / 8222
1553 / 8222
1554 / 8222
1555 / 8222
1556 / 8222
1557 / 8222
1558 / 8222
1559 / 8222
1560 / 8222
1561 / 8222
1562 / 8222
1563 / 8222
1564 / 8222
1565 / 8222
1566 / 8222
1567 / 8222
1568 / 8222
1569 / 8222
1570 / 8222
1571 / 8222
1572 / 8222
1573 / 8222
1574 / 8222
1575 / 8222
1576 / 8222
1577 / 8222
1578 / 8222
1579 / 8222
1580 / 8222
1581 / 8222
1582 / 8222
1583 / 8222
1584 / 8222
1585 / 8222
1586 / 8222
1587 / 8222
1588 / 8222
1589 / 8222
1590 / 8222
1591 / 8222
1592 / 8222
1593 / 8222
1594 / 8222
1595 / 8222
1596 / 8222
1597 / 8222
1598 / 8222
1599

2214 / 8222
2215 / 8222
2216 / 8222
2217 / 8222
2218 / 8222
2219 / 8222
2220 / 8222
2221 / 8222
2222 / 8222
2223 / 8222
2224 / 8222
2225 / 8222
2226 / 8222
2227 / 8222
2228 / 8222
2229 / 8222
2230 / 8222
2231 / 8222
2232 / 8222
2233 / 8222
2234 / 8222
2235 / 8222
2236 / 8222
2237 / 8222
2238 / 8222
2239 / 8222
2240 / 8222
2241 / 8222
2242 / 8222
2243 / 8222
2244 / 8222
2245 / 8222
2246 / 8222
2247 / 8222
2248 / 8222
2249 / 8222
2250 / 8222
2251 / 8222
2252 / 8222
2253 / 8222
2254 / 8222
2255 / 8222
2256 / 8222
2257 / 8222
2258 / 8222
2259 / 8222
2260 / 8222
2261 / 8222
2262 / 8222
2263 / 8222
2264 / 8222
2265 / 8222
2266 / 8222
2267 / 8222
2268 / 8222
2269 / 8222
2270 / 8222
2271 / 8222
2272 / 8222
2273 / 8222
2274 / 8222
2275 / 8222
2276 / 8222
2277 / 8222
2278 / 8222
2279 / 8222
2280 / 8222
2281 / 8222
2282 / 8222
2283 / 8222
2284 / 8222
2285 / 8222
2286 / 8222
2287 / 8222
2288 / 8222
2289 / 8222
2290 / 8222
2291 / 8222
2292 / 8222
2293 / 8222
2294 / 8222
2295 / 8222
2296 / 8222
2297

2906 / 8222
2907 / 8222
2908 / 8222
2909 / 8222
2910 / 8222
2911 / 8222
2912 / 8222
2913 / 8222
2914 / 8222
2915 / 8222
2916 / 8222
2917 / 8222
2918 / 8222
2919 / 8222
2920 / 8222
2921 / 8222
2922 / 8222
2923 / 8222
2924 / 8222
2925 / 8222
2926 / 8222
2927 / 8222
2928 / 8222
2929 / 8222
2930 / 8222
2931 / 8222
2932 / 8222
2933 / 8222
2934 / 8222
2935 / 8222
2936 / 8222
2937 / 8222
2938 / 8222
2939 / 8222
2940 / 8222
2941 / 8222
2942 / 8222
2943 / 8222
2944 / 8222
2945 / 8222
2946 / 8222
2947 / 8222
2948 / 8222
2949 / 8222
2950 / 8222
2951 / 8222
2952 / 8222
2953 / 8222
2954 / 8222
2955 / 8222
2956 / 8222
2957 / 8222
2958 / 8222
2959 / 8222
2960 / 8222
2961 / 8222
2962 / 8222
2963 / 8222
2964 / 8222
2965 / 8222
2966 / 8222
2967 / 8222
2968 / 8222
2969 / 8222
2970 / 8222
2971 / 8222
2972 / 8222
2973 / 8222
2974 / 8222
2975 / 8222
2976 / 8222
2977 / 8222
2978 / 8222
2979 / 8222
2980 / 8222
2981 / 8222
2982 / 8222
2983 / 8222
2984 / 8222
2985 / 8222
2986 / 8222
2987 / 8222
2988 / 8222
2989

...

4973 / 8222
4974 / 8222
4975 / 8222
4976 / 8222
4977 / 8222
4978 / 8222
4979 / 8222
4980 / 8222
4981 / 8222
4982 / 8222
4983 / 8222
4984 / 8222
4985 / 8222
4986 / 8222
4987 / 8222
4988 / 8222
4989 / 8222
4990 / 8222
4991 / 8222
4992 / 8222
4993 / 8222
4994 / 8222
4995 / 8222
4996 / 8222
4997 / 8222
4998 / 8222
4999 / 8222
5000 / 8222
5001 / 8222
5002 / 8222
5003 / 8222
5004 / 8222
5005 / 8222
5006 / 8222
5007 / 8222
5008 / 8222
5009 / 8222
5010 / 8222
5011 / 8222
5012 / 8222
5013 / 8222
5014 / 8222
5015 / 8222
5016 / 8222
5017 / 8222
5018 / 8222
5019 / 8222
5020 / 8222
5021 / 8222
5022 / 8222
5023 / 8222
5024 / 8222
5025 / 8222
5026 / 8222
5027 / 8222
5028 / 8222
5029 / 8222
5030 / 8222
5031 / 8222
5032 / 8222
5033 / 8222
5034 / 8222
5035 / 8222
5036 / 8222
5037 / 8222
5038 / 8222
5039 / 8222
5040 / 8222
5041 / 8222
5042 / 8222
5043 / 8222
5044 / 8222
5045 / 8222
5046 / 8222
5047 / 8222
5048 / 8222
5049 / 8222
5050 / 8222
5051 / 8222
5052 / 8222
5053 / 8222
5054 / 8222
5055 / 8222
5056

5656 / 8222
5657 / 8222
5658 / 8222
5659 / 8222
5660 / 8222
5661 / 8222
5662 / 8222
5663 / 8222
5664 / 8222
5665 / 8222
5666 / 8222
5667 / 8222
5668 / 8222
5669 / 8222
5670 / 8222
5671 / 8222
5672 / 8222
5673 / 8222
5674 / 8222
5675 / 8222
5676 / 8222
5677 / 8222
5678 / 8222
5679 / 8222
5680 / 8222
5681 / 8222
5682 / 8222
5683 / 8222
5684 / 8222
5685 / 8222
5686 / 8222
5687 / 8222
5688 / 8222
5689 / 8222
5690 / 8222
5691 / 8222
5692 / 8222
5693 / 8222
5694 / 8222
5695 / 8222
5696 / 8222
5697 / 8222
5698 / 8222
5699 / 8222
5700 / 8222
5701 / 8222
5702 / 8222
5703 / 8222
5704 / 8222
5705 / 8222
5706 / 8222
5707 / 8222
5708 / 8222
5709 / 8222
5710 / 8222
5711 / 8222
5712 / 8222
5713 / 8222
5714 / 8222
5715 / 8222
5716 / 8222
5717 / 8222
5718 / 8222
5719 / 8222
5720 / 8222
5721 / 8222
5722 / 8222
5723 / 8222
5724 / 8222
5725 / 8222
5726 / 8222
5727 / 8222
5728 / 8222
5729 / 8222
5730 / 8222
5731 / 8222
5732 / 8222
5733 / 8222
5734 / 8222
5735 / 8222
5736 / 8222
5737 / 8222
5738 / 8222
5739

6342 / 8222
6343 / 8222
6344 / 8222
6345 / 8222
6346 / 8222
6347 / 8222
6348 / 8222
6349 / 8222
6350 / 8222
6351 / 8222
6352 / 8222
6353 / 8222
6354 / 8222
6355 / 8222
6356 / 8222
6357 / 8222
6358 / 8222
6359 / 8222
6360 / 8222
6361 / 8222
6362 / 8222
6363 / 8222
6364 / 8222
6365 / 8222
6366 / 8222
6367 / 8222
6368 / 8222
6369 / 8222
6370 / 8222
6371 / 8222
6372 / 8222
6373 / 8222
6374 / 8222
6375 / 8222
6376 / 8222
6377 / 8222
6378 / 8222
6379 / 8222
6380 / 8222
6381 / 8222
6382 / 8222
6383 / 8222
6384 / 8222
6385 / 8222
6386 / 8222
6387 / 8222
6388 / 8222
6389 / 8222
6390 / 8222
6391 / 8222
6392 / 8222
6393 / 8222
6394 / 8222
6395 / 8222
6396 / 8222
6397 / 8222
6398 / 8222
6399 / 8222
6400 / 8222
6401 / 8222
6402 / 8222
6403 / 8222
6404 / 8222
6405 / 8222
6406 / 8222
6407 / 8222
6408 / 8222
6409 / 8222
6410 / 8222
6411 / 8222
6412 / 8222
6413 / 8222
6414 / 8222
6415 / 8222
6416 / 8222
6417 / 8222
6418 / 8222
6419 / 8222
6420 / 8222
6421 / 8222
6422 / 8222
6423 / 8222
6424 / 8222
6425

7029 / 8222
7030 / 8222
7031 / 8222
7032 / 8222
7033 / 8222
7034 / 8222
7035 / 8222
7036 / 8222
7037 / 8222
7038 / 8222
7039 / 8222
7040 / 8222
7041 / 8222
7042 / 8222
7043 / 8222
7044 / 8222
7045 / 8222
7046 / 8222
7047 / 8222
7048 / 8222
7049 / 8222
7050 / 8222
7051 / 8222
7052 / 8222
7053 / 8222
7054 / 8222
7055 / 8222
7056 / 8222
7057 / 8222
7058 / 8222
7059 / 8222
7060 / 8222
7061 / 8222
7062 / 8222
7063 / 8222
7064 / 8222
7065 / 8222
7066 / 8222
7067 / 8222
7068 / 8222
7069 / 8222
7070 / 8222
7071 / 8222
7072 / 8222
7073 / 8222
7074 / 8222
7075 / 8222
7076 / 8222
7077 / 8222
7078 / 8222
7079 / 8222
7080 / 8222
7081 / 8222
7082 / 8222
7083 / 8222
7084 / 8222
7085 / 8222
7086 / 8222
7087 / 8222
7088 / 8222
7089 / 8222
7090 / 8222
7091 / 8222
7092 / 8222
7093 / 8222
7094 / 8222
7095 / 8222
7096 / 8222
7097 / 8222
7098 / 8222
7099 / 8222
7100 / 8222
7101 / 8222
7102 / 8222
7103 / 8222
7104 / 8222
7105 / 8222
7106 / 8222
7107 / 8222
7108 / 8222
7109 / 8222
7110 / 8222
7111 / 8222
7112

7713 / 8222
7714 / 8222
7715 / 8222
7716 / 8222
7717 / 8222
7718 / 8222
7719 / 8222
7720 / 8222
7721 / 8222
7722 / 8222
7723 / 8222
7724 / 8222
7725 / 8222
7726 / 8222
7727 / 8222
7728 / 8222
7729 / 8222
7730 / 8222
7731 / 8222
7732 / 8222
7733 / 8222
7734 / 8222
7735 / 8222
7736 / 8222
7737 / 8222
7738 / 8222
7739 / 8222
7740 / 8222
7741 / 8222
7742 / 8222
7743 / 8222
7744 / 8222
7745 / 8222
7746 / 8222
7747 / 8222
7748 / 8222
7749 / 8222
7750 / 8222
7751 / 8222
7752 / 8222
7753 / 8222
7754 / 8222
7755 / 8222
7756 / 8222
7757 / 8222
7758 / 8222
7759 / 8222
7760 / 8222
7761 / 8222
7762 / 8222
7763 / 8222
7764 / 8222
7765 / 8222
7766 / 8222
7767 / 8222
7768 / 8222
7769 / 8222
7770 / 8222
7771 / 8222
7772 / 8222
7773 / 8222
7774 / 8222
7775 / 8222
7776 / 8222
7777 / 8222
7778 / 8222
7779 / 8222
7780 / 8222
7781 / 8222
7782 / 8222
7783 / 8222
7784 / 8222
7785 / 8222
7786 / 8222
7787 / 8222
7788 / 8222
7789 / 8222
7790 / 8222
7791 / 8222
7792 / 8222
7793 / 8222
7794 / 8222
7795 / 8222
7796

8222